In [1]:
# load libraries
library(DESeq2)
library(enrichplot)
library(clusterProfiler)
library(ggplot2)
library(org.Hs.eg.db)
library(gridExtra)
library(grid)
source('../utils/utils.R')

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, saveRDS, setdiff,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb



### load genes

In [2]:
# load genes used for NGS2025 (less columns, made from ../../ref/h38_e110_new/genes_new.gtf)
genes <- readRDS('../../rds/NGS-20250519/genes.rds')
genes

,chr,start,end,strand,gene_id,gene_name,gene_biotype
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
ENSG00000279928,1,182696,184174,+,ENSG00000279928,DDX11L17,unprocessed_pseudogene
ENSG00000228037,1,2581560,2584533,+,ENSG00000228037,NA,lncRNA
ENSG00000142611,1,3069168,3438621,+,ENSG00000142611,PRDM16,protein_coding
ENSG00000284616,1,5301928,5307394,-,ENSG00000284616,NA,lncRNA
ENSG00000157911,1,2403964,2413797,-,ENSG00000157911,PEX10,protein_coding
ENSG00000269896,1,2350414,2352820,-,ENSG00000269896,NA,transcribed_processed_pseudogene
ENSG00000228463,1,257864,359681,-,ENSG00000228463,NA,transcribed_processed_pseudogene
ENSG00000260972,1,5492978,5494674,+,ENSG00000260972,NA,lncRNA
ENSG00000224340,1,10054445,10054781,-,ENSG00000224340,RPL21P21,processed_pseudogene


In [22]:
# load genes used for 2023-2024 (using gut_genes, since brain's one was confused)
#ggenes = read.csv('../../output/gut/gut_genes.csv',row.names = 1)
#ggenes

,chr,start,end,strand,gene_id,gene_name,gene_biotype
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
ENSG00000279928,1,182696,184174,+,ENSG00000279928,DDX11L17,unprocessed_pseudogene
ENSG00000228037,1,2581560,2584533,+,ENSG00000228037,NA,lncRNA
ENSG00000142611,1,3069168,3438621,+,ENSG00000142611,PRDM16,protein_coding
ENSG00000284616,1,5301928,5307394,-,ENSG00000284616,NA,lncRNA
ENSG00000157911,1,2403964,2413797,-,ENSG00000157911,PEX10,protein_coding
ENSG00000269896,1,2350414,2352820,-,ENSG00000269896,NA,transcribed_processed_pseudogene
ENSG00000228463,1,257864,359681,-,ENSG00000228463,NA,transcribed_processed_pseudogene
ENSG00000260972,1,5492978,5494674,+,ENSG00000260972,NA,lncRNA
ENSG00000224340,1,10054445,10054781,-,ENSG00000224340,RPL21P21,processed_pseudogene


### load samples: all_samples (18 brain, 30 gut), 2025_samples (10 brain, 6 gut), 2024_samples (8 brain, 24 gut)

In [24]:
#write.table(samples,
#            file = "all_samples.tsv",
#            sep = "\t",              # табуляция как разделитель
#            quote = FALSE,           # не оборачивать строки в кавычки
#            row.names = FALSE,       # не сохранять индексы строк
#            col.names = TRUE)        # записать заголовки столбцов

In [3]:
all_samples =read.table("all_samples.tsv",
                      header = TRUE,
                      sep = "\t",
                      check.names = FALSE,
                      stringsAsFactors = FALSE)
all_samples

id,condition,replicate,tissue,organoid stage,organoid line,timepoint,name
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
S1,mock,s1,gut,undiff,duo363,24h,mock_gut_undiff_duo363_s1_S1
S10,VA1,s3,gut,undiff,duo363,24h,VA1_gut_undiff_duo363_s3_S10
S11,HAstV4,s2,gut,undiff,duo363,24h,HAstV4_gut_undiff_duo363_s2_S11
S12,HAstV4,s3,gut,undiff,duo363,24h,HAstV4_gut_undiff_duo363_s3_S12
S13,mock,s1,gut,diff,duo363,24h,mock_gut_diff_duo363_s1_S13
S14,mock,s2,gut,diff,duo363,24h,mock_gut_diff_duo363_s2_S14
S15,mock,s3,gut,diff,duo363,24h,mock_gut_diff_duo363_s3_S15
S16,MLB2,s1,gut,diff,duo363,24h,MLB2_gut_diff_duo363_s1_S16
S17,MLB2,s2,gut,diff,duo363,24h,MLB2_gut_diff_duo363_s2_S17


In [6]:
samples_2025 = read.table("../../src/NGS-20250519/samples.tsv",header = T,check.names = F)
rownames(samples_2025) = samples_2025$id
samples_2025

,condition,replicate,tissue,organoid.stage,organoid.line,timepoint,name,id
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
VL11,MLB2,s1,brain,NA,NA,10d,MLB2_brain_s1_10d_VL11,VL11
VL12,MLB2,s2,brain,NA,NA,10d,MLB2_brain_s2_10d_VL12,VL12
VL13,VA1,s1,brain,NA,NA,10d,VA1_brain_s1_10d_VL13,VL13
VL14,VA1,s2,brain,NA,NA,10d,VA1_brain_s2_10d_VL14,VL14
VL15,HAstV4,s1,brain,NA,NA,10d,HAstV4_brain_s1_10d_VL15,VL15
VL16,HAstV4,s2,brain,NA,NA,10d,HAstV4_brain_s2_10d_VL16,VL16
VL17,mock,s1,brain,NA,NA,10d,mock_brain_s1_10d_VL17,VL17
VL18,mock,s2,brain,NA,NA,10d,mock_brain_s2_10d_VL18,VL18
VL19,mock_trypsin,s1,brain,NA,NA,10d,mock_trypsin_brain_s1_10d_VL19,VL19


In [7]:
samples_2024 = read.table('../../src/both/both_samples_condition.tsv')

#bsamples <- samples %>% filter(tissue == "brain")
#gsamples <- samples %>% filter(tissue == "gut")
#gundiffsamples <- samples %>% filter(tissue == "gut", organoid_stage == "undiff")
#gdiffsamples <- samples %>% filter(tissue == "gut", organoid_stage == "diff")

samples_2024

,condition,replicate,organoid_stage,id,name,tissue
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
VL1,MLB2,1,4d,VL1,MLB2_brain_4d_VL1,brain
VL2,MLB2,2,4d,VL2,MLB2_brain_4d_VL2,brain
VL3,VA1,1,4d,VL3,VA1_brain_4d_VL3,brain
VL4,VA1,2,4d,VL4,VA1_brain_4d_VL4,brain
VL5,HAstV4,1,4d,VL5,HAstV4_brain_4d_VL5,brain
VL6,HAstV4,2,4d,VL6,HAstV4_brain_4d_VL6,brain
VL7,Mock,1,4d,VL7,Mock_brain_4d_VL7,brain
VL8,Mock,2,4d,VL8,Mock_brain_4d_VL8,brain
S22,HAstV4,1,diff,S22,HAstV4_gut_diff_S22,gut


### load counts (2025_all_counts, 2025_pc_counts, 2024_gut_all_counts, 2024_gut_pc_counts, 2024_brain_all_counts, 2024_brain_pc_counts)

In [8]:
#load all counts from NGS-20250519 - brain+gut
all_counts_2025 = as.matrix(read.csv('../../output/NGS-20250519/counts_analysis/counts_all_genes.csv',row.names = 1))
dim(all_counts_2025)

[1] 62757    16

In [16]:
#load all protein_coding counts from NGS-20250519 - brain+gut
pc_counts_2025 = as.matrix(read.csv('../../output/NGS-20250519/counts_protein_coding.csv',row.names = 1))
dim(pc_counts_2025)

[1] 20070    16

In [9]:
#load all counts from 2023-2024 - gut
gut_all_counts_2024 = as.matrix(read.csv('../../output/gut/counts_all_genes.csv',row.names = 1))
dim(gut_all_counts_2024)

[1] 62757    24

In [18]:
#load all protein_coding counts from 2023-2024 - gut
gut_pc_counts_2024 = as.matrix(read.csv('../../output/gut/counts_protein_coding.csv',row.names = 1))
dim(gut_pc_counts_2024)

[1] 20070    24

In [10]:
#load all counts from 2023-2024 - brain 
brain_all_counts_2024 = as.matrix(read.csv('../../output/brain/counts_all_genes.csv',row.names = 1))
dim(brain_all_counts_2024)

[1] 62757     8

In [20]:
#load all protein_coding counts from 2023-2024 - brain 
brain_pc_counts_2024 = as.matrix(read.csv('../../output/brain/counts_protein_coding.csv',row.names = 1))
dim(brain_pc_counts_2024)

[1] 20070     8

### load deseq results (2025_brain, 2024_brain, 2024_gundiff, 2024_gdiff)

In [24]:
# load deseq data from NGS-20250519 - brain
brain_deseq_2025 = readRDS('../../rds/NGS-20250519/deseq2_brain_all comparison.rds')
gut_deseq_2025 = readRDS('../../rds/NGS-20250519/deseq2_gut_MLB2_mock.rds')

In [25]:
# load deseq data from 2023-2024 - brain, gundiff, gdiff
brain_deseq_2024 = readRDS('../../rds/deseq2_brain_mockvirus.rds')
gundiff_deseq_2024 = readRDS('../../rds/deseq2_gut_undiff_mockvirus.rds')
gdiff_deseq_2024 = readRDS('../../rds/deseq2_gut_diff_mockvirus.rds')